In [222]:
import os
from sklearn import (datasets, pipeline, feature_extraction, svm,model_selection,naive_bayes, linear_model,metrics)
import spacy
import re
import string
import unidecode
from spacy.lang.es import Spanish
import numpy as np

parser = Spanish()
nlp = spacy.load("es")

In [112]:
data_directory = "independencia_catalunya"

dataset = datasets.load_files(data_directory, shuffle = True, random_state = 42, encoding = 'utf-8')

print("Data loaded.\n Classes: {classes}\nData: {datapoints}".format(
    classes = dataset.target_names,
    datapoints = len(dataset.data)
))

Data loaded.
 Classes: ['against', 'favor', 'neutral']
Data: 4319


In [250]:
def spacy_tokenizer(sentence):
   # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)
    
    # Lemmatizing each token and converting each token into lowercase
    #mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    # mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    mytokens = [ word.text for word in mytokens]

    # return preprocessed list of tokens
    return mytokens

In [146]:
def clean_corpus(corpus):
    processed_corpus = corpus.replace('\r \n','')
    
    processed_corpus = processed_corpus.lower()
    processed_corpus = processed_corpus.replace('#27s','')
    processed_corpus = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', '',processed_corpus)
    processed_corpus = re.sub('[!¡,¿?#@:\'".=´“”();-]|[0-9](%|h)?','',processed_corpus)
    processed_corpus = unidecode.unidecode(processed_corpus) # Elimina las ñ
    
    processed_corpus = processed_corpus.replace('+',' ')
    processed_corpus = processed_corpus.replace('km²','')
    processed_corpus = processed_corpus.replace('&gt','')
    
    processed_corpus = re.sub(r' +',' ',processed_corpus)
    # print(processed_corpus)
    return processed_corpus
    
    

In [141]:
clean_corpus(dataset.data[0])

me encanta como nos dan lecciones los que ni tan siquiera respetan sus programas electorales votantes pedrosanchez pp 


In [143]:
#i=0
#for w in dataset.data:
    #w = w.replace('\r \n','')
    #print('-'*20)
    #print(i)
    #print(dataset.target_names[dataset.target[i]])
    #print('{}\n'.format(w))
    #i+=1

In [148]:
processed_corpus = [clean_corpus(sw) for sw in dataset.data]



me encanta como nos dan lecciones los que ni tan siquiera respetan sus programas electorales votantes pedrosanchez pp 


In [220]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    processed_corpus, dataset.target, test_size = 0.2, random_state=42)

In [280]:
spacy_stopwords = spacy.lang.es.stop_words.STOP_WORDS

model = pipeline.Pipeline([
    ('counts',feature_extraction.text.CountVectorizer(
        lowercase = True,
        tokenizer = spacy_tokenizer,
        min_df = 5,
        ngram_range = (1,3),
        stop_words = spacy_stopwords
    )),
    ('tfidf', feature_extraction.text.TfidfTransformer()),
     ('LogisticRegression', linear_model.LogisticRegression())
    #('svm', svm.LinearSVC())
])

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('The model acurracy is {}'.format(
    model.score(X_test,y_test)))

The model acurracy is 0.7453703703703703


In [255]:
print(metrics.classification_report(y_test,y_pred, target_names = dataset.target_names))

              precision    recall  f1-score   support

     against       0.80      0.54      0.64       290
       favor       0.93      0.21      0.34        68
     neutral       0.73      0.93      0.82       506

   micro avg       0.75      0.75      0.75       864
   macro avg       0.82      0.56      0.60       864
weighted avg       0.77      0.75      0.72       864

